# Infer RFHLTH for unseen data
In this notebook we demonstrate inference on unseen data. In this case we use synthetic data but could be new data for real individuals. 

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PopHealthRisk").getOrCreate()

In [0]:
from pyspark.ml.classification import RandomForestClassificationModel

loaded_model = RandomForestClassificationModel.load("/Volumes/pophealthrisk/pophealthrisk/pophealthrisk/models/rf-1")

In [0]:
# Note that loading via MLflow does not work on serverless compute, but if set up correctly, we could load like this 
#import mlflow.spark

#loaded_model = mlflow.spark.load_model("models:/pophealthrisk.pophealthrisk.phr_rf@prod")

In [0]:
# Load synthetic data and assemble for inference
from pyspark.ml.feature import VectorAssembler

df = spark.read.parquet('/Volumes/pophealthrisk/pophealthrisk/pophealthrisk/synthetic_data.parquet', header=True, inferSchema=True)
feature_cols=["_AGEG5YR_clean","EDUCA_clean",'_BMI5', '_SMOKER3_clean', 'DRNKANY6_clean','INCOME3_clean','num_conditions','income_adj_pov']

assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features"
)

df_assembled = assembler.transform(df)

In [0]:
# Infer
predictions = loaded_model.transform(df_assembled)

In [0]:
# write prediction labels
predictions.select('prediction').write.parquet('/Volumes/pophealthrisk/pophealthrisk/pophealthrisk/synthetic_data_inferred.parquet')

{"ts": "2025-10-14 22:03:23.785", "level": "ERROR", "logger": "pyspark.sql.connect.logging", "msg": "GRPC Error received", "context": {}, "exception": {"class": "_MultiThreadedRendezvous", "msg": "<_MultiThreadedRendezvous of RPC that terminated with:\n\tstatus = StatusCode.INTERNAL\n\tdetails = \"[PATH_ALREADY_EXISTS] Path dbfs:/Volumes/pophealthrisk/pophealthrisk/pophealthrisk/synthetic_data_inferred.parquet already exists. Set mode as \"overwrite\" to overwrite the existing path. SQLSTATE: 42K04\"\n\tdebug_error_string = \"UNKNOWN:Error received from peer  {created_time:\"2025-10-14T22:03:23.784974582+00:00\", grpc_status:13, grpc_message:\"[PATH_ALREADY_EXISTS] Path dbfs:/Volumes/pophealthrisk/pophealthrisk/pophealthrisk/synthetic_data_inferred.parquet already exists. Set mode as \\\"overwrite\\\" to overwrite the existing path. SQLSTATE: 42K04\"}\"\n>", "stacktrace": [{"class": null, "method": "_execute_and_fetch_as_iterator", "file": "/databricks/python/lib/python3.12/site-packag

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-5003092535762550>, line 2
      1 # write prediction labels
----> 2 predictions.select('prediction').write.parquet('/Volumes/pophealthrisk/pophealthrisk/pophealthrisk/synthetic_data_inferred.parquet')

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/readwriter.py:779, in DataFrameWriter.parquet(self, path, mode, partitionBy, compression)
    777     self.partitionBy(partitionBy)
    778 self._set_opts(compression=compression)
--> 779 self.format("parquet").save(path)

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/readwriter.py:703, in DataFrameWriter.save(self, path, format, mode, partitionBy, **options)
    701     self.format(format)
    702 self._write.path = path
--> 703 _, _, ei = self._spark.client.execute_command(
    704     self._write.command(self._spark.client)